<a href="https://colab.research.google.com/github/Neel7317/PySpark/blob/main/LR_Using_PySpark(mlLib).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=af4db91e7de6c03329428d34cdb348204fbdd791cc55d0555cdc2da3f40cae68
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [5]:
df=spark.read.csv('/content/test2.csv',header=True,inferSchema=True)

In [7]:
df.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



In [9]:
df=df.dropna()

In [10]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [11]:
from pyspark.ml.feature import VectorAssembler

In [15]:
feature_ass=VectorAssembler(inputCols=['age','Experience'],outputCol='OutCol')

In [16]:
output=feature_ass.transform(df)

In [17]:
output.show()

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|     OutCol|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [18]:
final_col=output.select('OutCol','Salary')

In [19]:
final_col.show()

+-----------+------+
|     OutCol|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [20]:
from pyspark.ml.regression import LinearRegression

In [21]:
#train test split

In [22]:
train,test=final_col.randomSplit([0.75,0.25])

In [23]:
model=LinearRegression(featuresCol="OutCol",labelCol="Salary")

In [24]:
model=model.fit(train)

In [25]:
#coefficient
model.coefficients

DenseVector([-263.7076, 1767.624])

In [26]:
model.intercept

19919.060052215827

In [27]:
results=model.evaluate(test)

In [28]:
results.predictions.show()

+----------+------+----------------+
|    OutCol|Salary|      prediction|
+----------+------+----------------+
|[29.0,4.0]| 20000|19342.0365535255|
+----------+------+----------------+



In [29]:
results.meanAbsoluteError,results.meanSquaredError

(657.9634464744995, 432915.8968966016)